In [1]:

import random
import copy
import time
import operator
import sys
import math
from functools import reduce
import numpy as np
import matplotlib.pyplot as plt
import sys
import bqplot
sys.setrecursionlimit(1000000)
from openpyxl import load_workbook
import numpy as np
from bqplot import pyplot as plt
from bqplot import LinearScale, Scatter, Axis, Figure, Lines
%matplotlib inline 


In [2]:
(ALPHA, BETA, RHO, Q) = (1.0, 2.0, 0.5, 100.0)
# 城市数，蚁群
ant_num = 50
iteration = 180
m=0
grid = []
time_wait=0.1
data = load_workbook('test_data.xlsx')
book_sheet=data.active
rows=book_sheet.rows
x=[]
y=[]
for i ,row in enumerate(rows):
    if not isinstance(row[0].value,int ) or not isinstance(row[1].value,int ) or not isinstance(row[2].value,int ) :
        continue
    g=[int(row[0].value),int(row[1].value),int(row[2].value)]
    x.append(int(row[1].value))
    y.append(int(row[2].value))#为之后画图做准备
    grid.append(g)
arr = np.array(grid)  # 存储入一个二维数组
m=arr.shape[0]
# %%
def calculate_dist(i,j):
    dis=math.sqrt(pow(arr[i][1]-arr[j][1],2)+pow(arr[i][2]-arr[j][2],2))
    return dis
city_num = m
arr1 = [[float('inf')] * (city_num+5)] * (city_num+5)
distance_graph = np.array(arr1)
for i, item in enumerate(arr):
    for  j ,item in enumerate(arr):
        distance_graph[i+1][j+1]=calculate_dist(i,j)
distance_graph_graph = distance_graph
pheromone_graph = [[1.0 for col in range(city_num+5)] for raw in range(city_num+5)]

In [3]:
def change_ref(path,flag):
    for i in range(len(path)):
        n_x,n_y=x[path[i]-1],y[path[i]-1]
        ref.x=np.append(ref.x,n_x)
        ref.y=np.append(ref.y,n_y)
        time.sleep(time_wait)
    n_x,n_y=x[path[0]-1],y[path[0]-1]
    ref.x=np.append(ref.x,n_x)
    ref.y=np.append(ref.y,n_y)
    time.sleep(1)
    if not flag:
        ref.x=[]
        ref.y=[]

In [4]:
# ----------- 蚂蚁 -----------
class Ant(object):

    # 初始化
    def __init__(self, ID):

        self.ID = ID  # ID
        self.__clean_data()  # 随机初始化出生点

    # 初始数据
    def __clean_data(self):

        self.path = []  # 当前蚂蚁的路径
        self.total_distance_graph = 0.0  # 当前路径的总距离
        self.move_count = 0  # 移动次数
        self.current_city = -1  # 当前停留的城市
        self.open_table_city = [True for i in range(city_num+5)]  # 探索城市的状态

        city_index = random.randint(1, city_num)  # 随机初始出生点
        self.current_city = city_index
        self.path.append(city_index)
        self.open_table_city[city_index] = False
        self.move_count = 1

    # 选择下一个城市
    def __choice_next_city(self):

        next_city = -1
        select_citys_prob = [0.0 for i in range(city_num+5)]  # 存储去下个城市的概率
        total_prob = 0.0

        # 获取去下一个城市的概率
        for i in range(1,city_num+1):
            if self.open_table_city[i]:
                try:
                    # 计算概率：与信息素浓度成正比，与距离成反比
                    select_citys_prob[i] = pow(pheromone_graph[self.current_city][i], ALPHA) * pow(
                        (1.0 / distance_graph_graph[self.current_city][i]), BETA)
                    total_prob += select_citys_prob[i]
                except ZeroDivisionError as e:
                    print('Ant ID: {ID}, current city: {current}, target city: {target}'.format(ID=self.ID,
                                                                                                current=self.current_city,
                                                                                                target=i))
                    sys.exit(1)

        # 轮盘选择城市
        if total_prob > 0.0:
            # 产生一个随机概率,0.0-total_prob
            temp_prob = random.uniform(0.0, total_prob)
            for i in range(1,city_num+1):
                if self.open_table_city[i]:
                    # 轮次相减
                    temp_prob -= select_citys_prob[i]
                    if temp_prob < 0.0:
                        next_city = i
                        break

        # 未从概率产生，顺序选择一个未访问城市
        if (next_city == -1):
            next_city = random.randint(1, city_num)
            while ((self.open_table_city[next_city]) == False):  # if==False,说明已经遍历过了
                next_city = random.randint(1, city_num)
        # 返回下一个城市序号
        return next_city

    # 计算路径总距离，存储于total_distance_graph中
    def __cal_total_distance_graph(self):

        temp_distance_graph = 0.0

        for i in range(1, city_num):
            start, end = self.path[i], self.path[i - 1]
            temp_distance_graph += distance_graph_graph[start][end]

        # 回路
        end = self.path[0]
        temp_distance_graph += distance_graph_graph[start][end]
        self.total_distance_graph = temp_distance_graph

    # 移动操作
    def __move(self, next_city):

        self.path.append(next_city)
        self.open_table_city[next_city] = False
        self.total_distance_graph += distance_graph_graph[self.current_city][next_city]
        self.current_city = next_city
        self.move_count += 1

    # 搜索路径
    def search_path(self):

        # 初始化数据
        self.__clean_data()

        # 搜素路径，遍历完所有城市为止
        while self.move_count < city_num:
            # 移动到下一个城市
            next_city = self.__choice_next_city()
            self.__move(next_city)

        # 计算路径总长度
        self.__cal_total_distance_graph()


# ----------- TSP问题 -----------

In [5]:
class TSP(object):

    def __init__(self, n=city_num):

        # 城市数目初始化为city_num
        self.n = n
        self.new()
    def new(self, evt=None):
         # 初始城市之间的距离和信息素
        for i in range(city_num):
            for j in range(city_num):
                pheromone_graph[i][j] = 1.0

        self.ants = [Ant(ID) for ID in range(ant_num)]  # 初始蚁群
        self.best_ant = Ant(-1)  # 初始最优解
        self.best_ant.total_distance_graph = 1 << 31  # 初始最大距离
        self.iter = 1  # 初始化迭代次数
# 开始搜索
    def search_path(self, evt=None):
        while ((self.iter<iteration)):
            # 遍历每一只蚂蚁
            for ant in self.ants:
                # 搜索一条路径
                ant.search_path()
                # 与当前最优蚂蚁比较
                if ant.total_distance_graph < self.best_ant.total_distance_graph:
                    # 更新最优解
                    self.best_ant.total_distance_graph=ant.total_distance_graph
                    self.best_ant = copy.deepcopy(ant)
            # 更新信息素
            self.__update_pheromone_gragh()
            print(u"迭代次数：", self.iter, u"最佳路径总距离：", self.best_ant.total_distance_graph)
            self.iter += 1
            if (self.iter+1)%20==0:
                if  (self.iter+1)<iteration:
                    fig.title='After {0} iterations:最近距离为:{1}'.format(self.iter+1,self.best_ant.total_distance_graph)
                else:
                    fig.title="The final path:最近距离为:{0}".format(self.best_ant.total_distance_graph)
                change_ref(self.best_ant.path,(self.iter+1)>=iteration)
        for i in range(0,city_num):
            print(self.best_ant.path[i],"-> ",end='')
        print(self.best_ant.path[0])
    def __update_pheromone_gragh(self):

        # 获取每只蚂蚁在其路径上留下的信息素
        temp_pheromone = [[0.0 for col in range(city_num+5)] for raw in range(city_num+5)]
        for ant in self.ants:
            for i in range(1, city_num):
                start, end = ant.path[i - 1], ant.path[i]
                # 在路径上的每两个相邻城市间留下信息素，与路径总距离反比
                temp_pheromone[start][end] += Q / ant.total_distance_graph
                temp_pheromone[end][start] = temp_pheromone[start][end]

        # 更新所有城市之间的信息素，旧信息素衰减加上新迭代信息素
        for i in range(1,city_num+1):
            for j in range(1,city_num+1):
                pheromone_graph[i][j] = pheromone_graph[i][j] * RHO + temp_pheromone[i][j]
       
    # 主循环
    def mainloop(self):
        self.new()
        self.search_path()


In [6]:
# ----------- 程序的入口处 -----------

if __name__ == '__main__':
    print(u""" 
--------------------------------------------------------
    程序：蚁群算法解决TPS问题程序 
-------------------------------------------------------- 
    """)
    x_sc = LinearScale()
    y_sc = LinearScale()
    ref = Lines(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
    scatter = Scatter(x=x, y=y, 
                scales={'x': x_sc, 'y': y_sc},
                colors=['DarkOrange'], stroke='red', 
                stroke_width=0.4, default_size=20)
    x_ax = Axis(label='X', scale=x_sc)
    y_ax = Axis(label='Y', scale=y_sc, orientation='vertical')

    x_ax.min = 0
    x_ax.max = 2000
    fig = Figure(marks=[ref,scatter], title='Beginning', axes=[x_ax, y_ax],
                animation_duration=1000)
    display(fig)
    TSP().mainloop()


 
--------------------------------------------------------
    程序：蚁群算法解决TPS问题程序 
-------------------------------------------------------- 
    


Figure(animation_duration=1000, axes=[Axis(label='X', scale=LinearScale()), Axis(label='Y', orientation='verti…

迭代次数： 1 最佳路径总距离： 14851.501886291537
迭代次数： 2 最佳路径总距离： 12762.663698728315
迭代次数： 3 最佳路径总距离： 12072.81241842811
迭代次数： 4 最佳路径总距离： 12066.247672104653
迭代次数： 5 最佳路径总距离： 10954.666115233284
迭代次数： 6 最佳路径总距离： 10224.337985090331
迭代次数： 7 最佳路径总距离： 9214.80365833105
迭代次数： 8 最佳路径总距离： 8456.248199749178
迭代次数： 9 最佳路径总距离： 8456.248199749178
迭代次数： 10 最佳路径总距离： 8456.248199749178
迭代次数： 11 最佳路径总距离： 8456.248199749178
迭代次数： 12 最佳路径总距离： 8456.248199749178
迭代次数： 13 最佳路径总距离： 8236.434736966032
迭代次数： 14 最佳路径总距离： 8174.874086063603
迭代次数： 15 最佳路径总距离： 8174.874086063603
迭代次数： 16 最佳路径总距离： 8174.874086063603
迭代次数： 17 最佳路径总距离： 8174.874086063603
迭代次数： 18 最佳路径总距离： 8174.874086063603
